In [8]:
import pandas as pd

data = pd.read_csv("/content/hasil_text_preprocessing.csv")
data.head()

,Review,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,Clash of clans sekarang sering gangguan atau l...,Clash of clans sekarang sering gangguan atau l...,clash of clans sekarang sering gangguan atau l...,clash of clans sekarang sering gangguan atau l...,"['clash', 'of', 'clans', 'sekarang', 'sering',...","['gangguan', 'lose', 'connect', 'sinyalnya', '...",gangguan lose connect sinyalnya bagus ngebug b...,-10,negative
1,Buat kalian yang sering force close app periks...,Buat kalian yang sering force close app periks...,buat kalian yang sering force close app periks...,buat kalian yang sering force close app periks...,"['buat', 'kalian', 'yang', 'sering', 'force', ...","['force', 'close', 'app', 'periksa', 'device',...",force close app periksa device pengaruh app be...,-30,negative
2,kenapa sistem koneksi klien dengan server tida...,kenapa sistem koneksi klien dengan server tida...,kenapa sistem koneksi klien dengan server tida...,kenapa sistem koneksi klien dengan server tida...,"['kenapa', 'sistem', 'koneksi', 'klien', 'deng...","['sistem', 'koneksi', 'klien', 'server', 'diru...",sistem koneksi klien server dirubah terputus k...,-12,negative
3,gamenya bagus dan saya sudah memainkannya sang...,gamenya bagus dan saya sudah memainkannya sang...,gamenya bagus dan saya sudah memainkannya sang...,game bagus dan saya sudah memainkannya sangat ...,"['game', 'bagus', 'dan', 'saya', 'sudah', 'mem...","['bagus', 'memainkannya', 'saran', 'tolong', '...",bagus memainkannya saran tolong disconnect men...,-26,negative
4,"Kendala saya cuma di multipemain nya aja, soal...",Kendala saya cuma di multipemain nya aja soaln...,kendala saya cuma di multipemain nya aja soaln...,terhalang saya cuma di multipemain nya aja soa...,"['terhalang', 'saya', 'cuma', 'di', 'multipema...","['terhalang', 'multipemain', 'nyari', 'lawan',...",terhalang multipemain nyari lawan diatas balai...,-13,negative


In [9]:
data = data.dropna()

# Menggunakan Algoritma Deep learning

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text_akhir'])
X1 = tokenizer.texts_to_sequences(data['text_akhir'])
X1 = pad_sequences(X1)

y1 = data['polarity']
label_encoder = LabelEncoder()
y1 = label_encoder.fit_transform(y1)

y1 = to_categorical(y1, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=10, input_length=X1.shape[1]),
    SimpleRNN(16, return_sequences=False),
    Dropout(0,5),
    Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(X_train, y_train, epochs=5, batch_size=128)

train_accuracy = hist.history['accuracy'][-1]
print(f'Train Accuracy: {train_accuracy:.2f}')

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


810/810 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.6264 - loss: 0.7982
Epoch 2/5
810/810 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.8943 - loss: 0.3092
Epoch 3/5
810/810 ━━━━━━━━━━━━━━━━━━━━ 40s 25ms/step - accuracy: 0.9435 - loss: 0.1914
Epoch 4/5
810/810 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.9566 - loss: 0.1463
Epoch 5/5
810/810 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9656 - loss: 0.1136
Train Accuracy: 0.96
810/810 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9006 - loss: 0.3160
Test Accuracy: 0.90


In [11]:
import joblib
model.save('model_rnn.keras')
joblib.dump(label_encoder, 'label_encoder.pkl')
joblib.dump(tokenizer, 'tokenizer.pkl')

['tokenizer.pkl']

# Skema Pelatihan Pertama

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X = data['text_akhir']
y = data['polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

tfidf = TfidfVectorizer(max_features=550, min_df=20, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')

svm_model.fit(X_train, y_train)

y_pred_train_svm = svm_model.predict(X_train)
y_pred_test_svm = svm_model.predict(X_test)

accuracy_train_svm = accuracy_score(y_pred_train_svm, y_train)
accuracy_test_svm = accuracy_score(y_pred_test_svm, y_test)

print('SVM - accuracy_test:', accuracy_train_svm)
print('SVM - accuracy_test:', accuracy_test_svm)

joblib.dump(svm_model, 'svm_model.pkl')
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

SVM - accuracy_test: 0.8614484582858688
SVM - accuracy_test: 0.8580144418272386


['label_encoder.pkl']

In [13]:
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

# Skema Pelatihan Kedua

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

X = data['text_akhir']
y = data['polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

tfidf = TfidfVectorizer(max_features=400, min_df=20, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

random_forest = RandomForestClassifier()

random_forest.fit(X_train, y_train)

y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9554186859228081
Random Forest - accuracy_test: 0.8096304591265397


# Skema Pelatihan Ketiga

In [16]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import numpy as np

X = data['text_akhir']
y = data['polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_tokenized = [text.split() for text in X]

w2v_model = Word2Vec(sentences=X_tokenized, vector_size=100, window=5, min_count=3, workers=4)

def vectorize_text(text):
    words = text.split()
    word_vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(w2v_model.vector_size)

X_vectors = np.array([vectorize_text(text) for text in X])

X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

svm_model_word2vec = SVC(kernel='rbf')

svm_model_word2vec.fit(X_train, y_train)

y_pred_train_svm = svm_model_word2vec.predict(X_train)
y_pred_test_svm = svm_model_word2vec.predict(X_test)

accuracy_train_svm = accuracy_score(y_pred_train_svm, y_train)
accuracy_test_svm = accuracy_score(y_pred_test_svm, y_test)

print('SVM - accuracy_test:', accuracy_train_svm)
print('SVM - accuracy_test:', accuracy_test_svm)

SVM - accuracy_test: 0.782219604965922
SVM - accuracy_test: 0.7772328841178515


# Skema Pelatihan Ke empat

In [17]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

X = data['text_akhir']
y = data['polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_tokenized = [text.split() for text in X]

w2v_model = Word2Vec(sentences=X_tokenized, vector_size=50, window=2, min_count=1, workers=2)

def vectorize_text(text):
    words = text.split()
    word_vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(w2v_model.vector_size)

X_vectors = np.array([vectorize_text(text) for text in X])

X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.3, random_state=42)

random_forest_word2vec = RandomForestClassifier()

random_forest_word2vec.fit(X_train, y_train)

y_pred_train_rf = random_forest_word2vec.predict(X_train)
y_pred_test_rf = random_forest_word2vec.predict(X_test)

accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9999889670998918
Random Forest - accuracy_test: 0.7370575363624662


In [19]:
joblib.dump(random_forest_word2vec, 'random_forest_word2vec.pkl')
w2v_model.save('word2vec_model.model')